In [10]:
import math

In [11]:
"""
SquareGrid class represents agent's environment
0 - cell is traversable
1 - cell is blocked
"""
class SquareGrid:
    #set width, height and fill the grid with zeroes (fully traversable grid)
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[0 for _ in range(width)] for _ in range(height)]
    
    #fill the grid given the input_map, e.g. the string with '#' standing for blocked cells
    def get_map(self, input_map):
        for i in range(self.height):
            for j in range(self.width):
                if input_map[i * self.width + j] == '#':
                    self.grid[i][j] = 1
    
    #out of bounds check
    def in_bounds(self, i, j):
        return 0 <= j < self.width and 0 <= i < self.height
    
    #blocked cell check
    def traversable(self, i, j):
        return not self.grid[i][j]
    
    
    def neighbors(self, i, j, diagonal=False, cutcorners=False, squeeze=False):
        """
        function, that returns neighbor nodes to the current node (i, j) according to the following parameters:
        diagonal: True, if diagonal moves are allowed
        cutcorners: True, if the agent is allowed to cut corners (only valid id diagoanl is True)
        squeeze: True, if the is also allowed to squeeze between obstacles (only valid when cutcornres is True)
        """
        neighbors = []
        #very primitive code (hard-code). works for cardinal moves only!
        if self.in_bounds(i, j - 1) and self.traversable(i, j - 1):
            neighbors.append((i, j - 1)) #move left
        if self.in_bounds(i - 1, j) and self.traversable(i - 1, j):
            neighbors.append((i - 1, j)) #move up
            
        if self.in_bounds(i, j + 1) and self.traversable(i, j + 1):
            neighbors.append((i, j + 1)) #move right
            
        if self.in_bounds(i + 1, j) and self.traversable(i + 1, j):
            neighbors.append((i + 1, j)) #move down
        
        return neighbors
    
        #HOMEWORK
        #REWRITE THE CODE to handle all the parameters (diagonal, curcorners, squeeze) and try to get read of the hard-code hooks.
        


In [12]:
#That's how we represent the map and convert it to SquareGrid object
test_map = '''
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . .  
. . . . . . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . . . . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . .
. . . . . . . . . . . . . c . . . . . . . . . . . . . . . .

'''

#define the SquareGrid object and fill it with a given map
test_grid = SquareGrid(30, 15) #make sure the dimensions match the drawn map
test_grid.get_map(test_map.translate({ ord(c): None for c in ' \n\t\r' })) #remove all whitespaces, tabs etc. 

#validate that map is converted correctly to the SquareGrid object
for gr in test_grid.grid:
    print(*gr)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [13]:
class Node:
    """
    Node class represents a search node
    i, j: coordinates of corresponding grid element
    g: g-value of the node
    h: h-value of the node
    F: f-value of the node
    parent: pointer to the parent-node 
    """
    def __init__(self, i, j, g=math.inf, h=math.inf, F=-1, parent=None):
        self.i = i
        self.j = j
        self.g = g
        if F==-1:
            self.F = self.g + h
        else:
            self.F = F        
        self.parent = parent

#### Now let's define a class for our OPEN list
Let's start withis a very primitive, straight-forward, non-efficent way to implement OPEN list.

In [14]:
class OpenListBasic:
    def __init__(self):
        self.elements = []
    
    #empty should infrom whether the OPEN is exhausted or not (in case it is - the search main loop should be interrupted)
    def empty(self):
        if len(self.elements) != 0:
            return False
        return True
    
    #get is the method that finds the best node (the one with the lowest  f-value),removes it from OPEN and returns it
    def get(self):
        best_F = math.inf
        best_coord = 0
        for i in range(len(self.elements)):
            if self.elements[i].F < best_F:
                best_coord = i
                best_F = self.elements[i].F
                
        # after we found the element with the lowest F value, we need to delete found element from OPEN list
        best = self.elements.pop(best_coord)
        # and return it
        return best
    
    #put is the method that puts (e.g. inserts or updates) the node to OPEN
    #When implementing it do not forget to handle all the possible situations:
    #- node already in OPEN but the new f-value is better;
    #- node already in OPEN but the new f-value is worse;
    #- node is not in OPEN.
    def put(self, item):
        for i in range(len(self.elements)):
            if self.elements[i].i == item.i and self.elements[i].j == item.j:
                if self.elements[i].F > item.F:
                    self.elements[i].F = item.F
                    return
                else:
                    return
                
        
        self.elements.append(item)
        return

Now let's implement more complicated Open List where all elements are sorted by F value

In [15]:
class OpenListSorted:
    def __init__(self):
        self.elements = []
    
    #empty should infrom whether the OPEN is exhausted or not (in case it is - the search main loop should be interrupted)
    def empty(self):
        if len(self.elements) != 0:
            return False
        return True
    
    #get is the method that finds the best node (the one with the lowest  f-value),
    # removes it from OPEN and returns it
    # in the sorted list we just need to take the first one
    def get(self):
        return self.elements.pop(0)
    
    #put is the method that puts (e.g. inserts or updates) the node to OPEN
    #When implementing it do not forget to handle all the possible situations:
    #- node already in OPEN but the new f-value is better;
    #- node already in OPEN but the new f-value is worse;
    #- node is not in OPEN.
    def put(self, item):
        position = 0
        position_found = False
        
        for i in range(len(self.elements)):
            # we will be looking for the right place for current element and 
            # check if the node is in OPEN at the same time
            if not position_found and self.elements[item.i][i].F >= item.F:
                position = i
                position_found = True
                
            if self.elements[i].i == item.i and self.elements[i].j == item.j:
                if item.F > self.elements[i].F:
                    return
                elif position == i:
                    self.elements[i] = item
                    return
                else:
                    self.elements[i].pop(position)
                    break
                
        self.elements.insert(position, item)
        return

Previous implementation is focused on faster access to element with the lowest F-value, but in order to check wheter element is in OPEN or not we have to look at every element in the list

Let's fix that

In [87]:
class OpenList:
    """
    This OpenList class represents a struct for efficient OPEN list representation
    """
    def __init__(self, height):
        self.elements = [[] for _ in range(height)]
        self.size = 0
        
    #empty should infrom whether the OPEN is exhausted or not (in case it is - the search main loop should be interrupted)
    def empty(self):
        for elem in self.elements:
            if len(elem) != 0:
                return False
        return True
    
    
    #get is the method that finds the best node (the one with the lowest  f-value),removes it from OPEN and returns it
    def get(self):
        best_F = math.inf
        best_coord = 0
        for coord in range(len(self.elements)):
            if len(self.elements[coord]) == 0:
                continue
            if self.elements[coord][0].F < best_F:
                best_coord = coord
                best_F = self.elements[coord][0].F
                
        # after we found the element with the lowest F value, we need to delete found element from OPEN list
        best = self.elements[best_coord].pop(0)
        # and return it
        return best
    
    #put is the method that puts (e.g. inserts or updates) the node to OPEN
    #When implementing it do not forget to handle all the possible situations:
    #- node already in OPEN but the new f-value is better;
    #- node already in OPEN but the new f-value is worse;
    #- node is not in OPEN.
    def put(self, item):
        if len(self.elements[item.i]) == 0:
            self.elements[item.i].append(item)
            self.size += 1
            return
        position = 0
        position_found = False
        
        #find the position on which to insert our new element
        # meanwhile you should also look for the node with the same coordinates in the OPEN already
        
        for i in range(len(self.elements[item.i])):
            if not position_found and self.elements[item.i][i].F >= item.F:
                position = i
                position_found = True
                    
            if self.elements[item.i][i].j == item.j:
                if item.F > self.elements[item.i][i].F:
                    return
                elif position == i:
                    self.elements[item.i][i].F = item.F
                    self.elements[item.i][i].g = item.g
                    self.elements[item.i][i].parent = item.parent
                    return
                else:
                    self.elements[item.i].pop(position)
                    self.size -= 1
                    break
                    
        self.size += 1           
        self.elements[item.i].insert(position, item)
        return

#### Now let's define a class for our CLOSED list

It also can be really simple and we ca just use list for it

In [88]:
class ClosedListBasic:
    def __init__(self, height):
        self.elements = []
        
    def length(self):
        return len(self.elements)
        
    def exists(self, item):
        return item  in self.elements
    
    def put(self, item):
        self.elements.append(item)

And we can also make Closed list more efficient

In [89]:
class ClosedList:
    def __init__(self, width):
        self.elements = dict()
        self.width = width
        
    def length(self):
        return len(self.elements)
        
    def exists(self, item):
        return item.i * self.width + item.j in self.elements
    
    def put(self, item):
        self.elements[item.i * self.width + item.j] = item

### Heuristics for grid maps

#### Euclidean distance

Straight line distance is the most intuitive thing to think of as it is the true shortest distance on a plane (thus 100% admissable). Calculation involves taking the square root (so it's a "slow" heuristic). Moreover this distance is not accurate for grid-worlds in many cases as the agent is limited to cardinal and diagonal moves only.

In [90]:
def euclidean_heuristic(a_i, a_j, b_i, b_j):
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return math.sqrt(dx * dx + dy * dy)

#### Manhattan distance

More accurate heuristic when cardinal moves are allowed. It is not an admissable heuristic if diagonal moves are allowed as well.

In [91]:
def manhattan_distance(a_i, a_j, b_i, b_j):
    #HOMEWORK
    #REWRITE THE CODE (it's Euclidean distance now)
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return math.sqrt(dx * dx + dy * dy)

#### Diagonal distance

More accurate heuristic when diagonal moves are allowed.

In [92]:
def diagonal_heuristic(a_i, a_j, b_i, b_j):
    #HOMEWORK
    #REWRITE THE CODE (it's Euclidean distance now)
    dx = abs(a_i - b_i)
    dy = abs(a_j - b_j)
    return math.sqrt(dx * dx + dy * dy)

__Let's start with A*__

In [93]:
def calculate_heuristic(a_i, a_j, goal_i, goal_j, heuristic_type='euclid', alpha=1):
    if heuristic_type == 'euclidean':
        return euclidean_heuristic(a_i, a_j, goal_i, goal_j)
    if heuristic_type == 'octile':
        return diagonal_heuristic(a_i, a_j, goal_i, goal_j)
    if heuristic_type == 'manhattan':
        return manhattan_heuristic(a_i, a_j, goal_i, goal_j) 
    
def calculate_cost(a_i, a_j, b_i, b_j):
    return math.sqrt(abs(a_i - b_i) * abs(a_i - b_i) + abs(a_j - b_j) * abs(a_j - b_j))


def search(grid, start_i, start_j, goal_i, goal_j,
           heuristic_type='euclidean',
           heuristic_weight=1,
           diagonal=False, 
           cutcorners=False, 
           squeeze=False):
    
    OPEN = OpenList(grid.height)
    #OPEN = OpenListBasic()
    start_node = Node(start_i, start_j, 0, 
                      calculate_heuristic(start_i, start_j, goal_i, goal_j, heuristic_type))
    OPEN.put(start_node)
    CLOSE = ClosedList(grid.width)
    
    while not OPEN.empty():
        current = OPEN.get() #retrieve the best search node from OPEN
        CLOSE.put(current) #put the node to CLOSE
        
        if current.i == goal_i and current.j == goal_j:
            print("Path has been found!")
            return current, CLOSE
        
        for (i, j) in grid.neighbors(current.i, current.j):
            if not CLOSE.exists(Node(i, j)):
                g_cur = current.g + calculate_cost(current.i, current.j, i, j)
                h_cur = calculate_heuristic(i, j, goal_i, goal_j, heuristic_type)
                f_cur = g_cur + heuristic_weight * h_cur
                new_node = Node(i, j, g_cur, h_cur, f_cur, current)
                OPEN.put(new_node)
                
    print("Path NOT found")
    return current, CLOSE

In [94]:
input_map = '''
. . . . . . . . . . . . . . . . . . . . . # # . . . . . . .  
. . . . . . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . . . . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . c . . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . # # # # # . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . # # . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . # # . . . . . . . . . . . . . . .
. . . . . . . . . . . . . c . . . . . . . . . . . . . . . .

'''

In [95]:
%%time

g = SquareGrid(30, 15)
g.get_map(input_map.translate({ ord(c):None for c in ' \n\t\r' }))
goal, CLOSE = search(g, 7, 7, 14, 29)

Path has been found!
CPU times: user 2.52 ms, sys: 70 µs, total: 2.59 ms
Wall time: 2.55 ms


In [96]:
#some valuable info
print("Path's lenght (largest g-value in case path NOT found):", goal.g)
print("Number of steps:", CLOSE.length())

Path's lenght (largest g-value in case path NOT found): 29.0
Number of steps: 83


In [97]:
def make_path(goal):
    current = goal
    path = []
    while current.parent:
        path.append((current.i, current.j))
        current = current.parent;
    path.append((current.i, current.j))
    return path[::-1]

path = make_path(goal)
print("Found path:", *path)

Found path: (7, 7) (7, 8) (8, 8) (9, 8) (10, 8) (11, 8) (12, 8) (13, 8) (13, 9) (13, 10) (13, 11) (13, 12) (14, 12) (14, 13) (14, 14) (14, 15) (14, 16) (14, 17) (14, 18) (14, 19) (14, 20) (14, 21) (14, 22) (14, 23) (14, 24) (14, 25) (14, 26) (14, 27) (14, 28) (14, 29)


In [98]:
def print_path(g, path):
    new_grid = g.grid.copy()
    for i in range(g.height):
        for j in range(g.width):
            if (i, j) in path:
                new_grid[i][j] = '*'
            else:
                new_grid[i][j] = g.grid[i][j]
    for gr in new_grid:
        print(*gr)


path = make_path(goal)
print_path(g, path)

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0
0 0 0 1 1 0 0 * * 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 * 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 * 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 * 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 1 1 0 0 0 * 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 * 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 * * * * * 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 * * * * * * * * * * * * * * * * * *


Now we will try our implementation on real map

In [106]:
with open('./maps/1.txt', 'r') as file:
    map1 = file.read()
    
map_grid = SquareGrid(512, 512) #make sure the dimensions match the drawn map
map_grid.get_map(map1.translate({ ord(c): None for c in ' \n\t\r' })) #remove all whitespaces, tabs etc. 

In [107]:
%%time
goal, CLOSE = search(map_grid, 510, 47, 10, 404)

Path has been found!
CPU times: user 33 s, sys: 99.5 ms, total: 33.1 s
Wall time: 33.2 s


In [108]:
#some valuable info
print("Path's lenght (largest g-value in case path NOT found):", goal.g)
print("Number of steps:", CLOSE.length())

Path's lenght (largest g-value in case path NOT found): 865.0
Number of steps: 128770


### Home work

1. Look for 'HOMEWORK' comments and follow them
2. Provide a short report (doc or pdf) on your experience with pathfinding with different heuristics and heuristics weight.
